In [1]:
import os
import librosa   #thư viện xủ lý tín hiệu âm thanh
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #để đọc file wav 
import warnings
import tensorflow as tf

c:\python 3.7\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\python 3.7\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\python 3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [2]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 6, 22, 8)          1160      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 6, 22, 8)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 11, 8)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 11, 16)         4624      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 3, 11, 16)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 80)               

In [34]:

import pyaudio
import wave
import struct
import matplotlib.pyplot as plt
import librosa.display
CHUNK = 11000 # read each 500 miliseconds
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 20
WAVE_OUTPUT_FILENAME = "output.wav"
# circular_buf_size = 50
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
frames = []
chromas = []
circular_index = 0
print("* recording")
inputDataNew = np.ndarray(shape=(CHUNK), dtype=float)
inputDataOld = np.ndarray(shape=(CHUNK), dtype=float)
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK) 
    #convert to float array
    for j in range(0, int(len(data)/4)-1):
        value = struct.unpack('f', data[j*4:j*4+4])
        inputDataNew[j] = value[0]
    #megre old and new
    inputData = np.concatenate((inputDataOld,inputDataNew))
    inputDataOld = inputDataNew.copy()
    if i<1:
        continue
    chroma = librosa.feature.chroma_stft(y=inputData, sr=16000)
    chromas.append(chroma)
    modelInput = np.array(chroma).reshape(1,12,43,1)
    output = model.predict(modelInput)
    indexMax = np.argmax(output[0], axis=0) 
#     if output[0][indexMax] >0.001:
    os.system( 'cls' )
    print ("time:",i," prediction:",indexMax," value:", output[0][indexMax])
#     plt.figure(figsize=(10, 4))
#     librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
#     plt.colorbar()
#     plt.title('Chromagram')
#     plt.tight_layout()
#     plt.show()
print("* done recording")    

    

* recording
time: 1  prediction: 10  value: 0.0056877756
time: 2  prediction: 2  value: 0.00026139617
time: 3  prediction: 5  value: 0.0017669797
time: 4  prediction: 0  value: 0.01600191
time: 5  prediction: 5  value: 0.0024671853
time: 6  prediction: 10  value: 0.008758573
time: 7  prediction: 10  value: 0.03422639
time: 8  prediction: 10  value: 0.03310286
time: 9  prediction: 10  value: 0.0037700369
time: 10  prediction: 10  value: 0.0036492606
time: 11  prediction: 0  value: 0.0064727664
time: 12  prediction: 0  value: 0.002619177
time: 13  prediction: 0  value: 0.0022180378
time: 14  prediction: 10  value: 0.0023910052
time: 15  prediction: 1  value: 0.003000021
time: 16  prediction: 5  value: 0.17327073
time: 17  prediction: 0  value: 0.0005425513
time: 18  prediction: 1  value: 0.0061828196
time: 19  prediction: 10  value: 0.007669927
time: 20  prediction: 10  value: 0.00082511286
time: 21  prediction: 1  value: 0.0018610656
time: 22  prediction: 10  value: 0.012144316
time: 23

In [32]:
chroma.shape


(12, 43)

In [5]:
import matplotlib.pyplot as plt
import librosa.display

plt.figure(figsize=(10, 4))
librosa.display.specshow(chromas[5], y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()
plt.show()

IndexError: list index out of range

<Figure size 720x288 with 0 Axes>

In [ ]:
len(inputData)

In [ ]:
(chroma.shape)

In [ ]:
samples